In [ ]:
import os


os.environ["GH_GQL_API_TOKEN"] = "YOUR_TOKEN_HERE"
print("Setting environment variable: %s" % os.getenv("GH_GQL_API_TOKEN"))



In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
from gql import gql, Client, AIOHTTPTransport
import asyncio
import nest_asyncio
from IPython.display import HTML, ProgressBar


# Let nested event loops in Notebook
nest_asyncio.apply()

TOP = 100
users = []

TOKEN = "bearer " + os.getenv("GH_GQL_API_TOKEN")

# Select your transport with a GitHub url endpoint
transport = AIOHTTPTransport(
    url="https://api.github.com/graphql", headers={"Authorization": TOKEN}
)

client = Client(
    transport=transport, fetch_schema_from_transport=True,
)


def progress(pb):
  "Display progress bar Ipython"
  print("\r", pb, sep="", end="", flush=True)


async def main():

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with client as session:
        # Provide a GraphQL query
        query = gql(
            """
            query search($query: String!, $type: SearchType!, $numOfResults: Int!, $nextPageCursor: String) {
              search(type: $type, query: $query, first: $numOfResults, after: $nextPageCursor) {
                pageInfo {
                  hasNextPage
                  endCursor
                }
                userCount
                nodes {
                  ... on User {
                    name
                    login
                    location
                    bio
                    avatarUrl
                    followers {
                      totalCount
                    }
                    contributionsCollection {
                      contributionCalendar{
                        totalContributions
                      }
                      totalCommitContributions
                      totalPullRequestContributions
                      restrictedContributionsCount
                    }
                  }
                }
              }
            }
        """
        )

        params = {"query": "location:Cuba", "type": "USER", "numOfResults": 20}

        print("Getting users...")

        result_users = []
        result_count = 0
        result_next = True
        first = True
        pb = ProgressBar(result_count)

        while result_next:
            try:
                result = await session.execute(query, variable_values=params)
            except asyncio.exceptions.TimeoutError:
                continue
            
            result_users += result.get("search").get("nodes", [])
            result_count = result.get("search").get("userCount", 0)

            if first:
                pb.total = result_count
                first = False

            pb._progress = len(result_users)
            result_next = result.get("search").get("pageInfo").get("hasNextPage", False)
            params["nextPageCursor"] = (
                result.get("search").get("pageInfo").get("endCursor")
            )
            progress(pb)
            await asyncio.sleep(1)

        # print(result)
        global users
        users = result_users
        print("\nTotal GitHub Users from Cuba: %s" % len(users))


asyncio.run(main())



Getting users...
[============================================================] 972/972
Total GitHub Users from Cuba: 972


In [3]:
for user in users:
    if user:
        user["followers"] = user.get("followers").get("totalCount", 0)
        user["contributions"] = user.get("contributionsCollection").get(
            "contributionCalendar"
        ).get("totalContributions", 0) - user.get(
            "contributionsCollection"
        ).get(
            "restrictedContributionsCount", 0
        )
        user["commits"] = user.get("contributionsCollection").get(
            "totalCommitContributions", 0
        ) + user.get("contributionsCollection").get(
            "totalPullRequestContributions", 0
        )
        del user["contributionsCollection"]

df = pd.DataFrame(users)
df.dropna(how='all', inplace=True)
# df = df.reset_index(drop=True)
df = df.sort_values(by="commits", ascending=False)

# Top Ten Cuba
new_dtypes = {
    "followers": np.int64,
    "contributions": np.int64,
    "commits": np.int64
}
position = ["\U0001F947", "\U0001F948", "\U0001F949"] + list(range(4, TOP + 1))

df_top_ten = df[:TOP]
df_top_ten = df_top_ten.astype(new_dtypes)
# Clean
df_top_ten.fillna("", inplace=True)
# Re-Order Columns
df_top_ten = df_top_ten[
    [
        "name",
        "login",
        "location",
        "commits",
        "contributions",
        "followers",
        "bio",
        "avatarUrl",
    ]
]
df_top_ten.insert(0, "#", position)
print("Top %s of %s GitHub Users from Cuba" % (TOP, len(users)))
print("Generated at: %s UTC" % datetime.utcnow())
HTML(df_top_ten.to_html(index=False))



Top 100 of 972 GitHub Users from Cuba
Generated at: 2020-09-07 16:37:48.932237 UTC


#,name,login,location,commits,contributions,followers,bio,avatarUrl
🥇,Maikel Llamaret Heredia,llamaret,Cuba,4554,4568,19,Creator of the SWL-X Project. Founder of Web & Media Integrated Solutions. Currently co administrator of the Web of the Users Group of Free Technologies in Cuba,https://avatars2.githubusercontent.com/u/27508028?u=b22f410e87d9cc1f91029193a72dc4791dda7b6b&v=4
🥈,Leynier Gutiérrez González,leynier,"La Habana, Cuba",2720,2991,42,Computer Science Student and Full Stack Developer,https://avatars1.githubusercontent.com/u/36774373?u=60eee7ab14aada5aab8af6fbd11d14732750a7ab&v=4
🥉,Yudivián Almeida Cruz,yudivian,"La Habana, Cuba",2034,2035,24,,https://avatars1.githubusercontent.com/u/5324359?v=4
4,Phaser Editor 2D,PhaserEditor2D,Cuba,1567,1593,30,Phaser Editor 2D official account.\r\n\r\nIDE for HTML5 2D game development.,https://avatars1.githubusercontent.com/u/12187299?u=f6f0d58173deee2333a45f0f24f2145324da3047&v=4
5,Guillermo,wil92,Cuba,1361,1613,11,happy coding!,https://avatars3.githubusercontent.com/u/10768089?u=132076faeebea896962d63696ad8d8c1ddfdfab9&v=4
6,Luis Antonio Correa Leyva,correaleyval,"Santiago de Cuba, Cuba",1103,1172,34,Computer Scientist,https://avatars2.githubusercontent.com/u/32603705?u=c1695bd968871f7671f04b327096d03e15d2b5f6&v=4
7,Alexis López Zubieta,azubieta,Cuba,1048,1101,54,"Professional software engineer, open source developer and supporter. C/C++ and Python coder.",https://avatars1.githubusercontent.com/u/1138094?u=24a26ed2fe8db89192c0190a5e33eabff8356b4a&v=4
8,Erich Garcia Cruz,n3omaster,Cuba,847,854,16,"PHP Developer, innovation lover, improvement lover, studying lover.",https://avatars0.githubusercontent.com/u/10359180?u=e787c69b83f9d6db25feef9a95bd1c243ffa9410&v=4
9,Jose Antonio Pereiro Morejon,josePereiro,"Havana, Cuba",790,820,0,,https://avatars3.githubusercontent.com/u/44244693?u=b721fe85c1ee1246aca1acb560cc1620d3d131ff&v=4
10,Asiel Díaz Benítez,adbenitez,Cuba,779,909,21,"Computer Scientist. Likes: bash, emacs, python, web, email, free/libre software, GNU project.",https://avatars2.githubusercontent.com/u/24558636?u=336dd09fcb3bb4ff147f3ae4082f6461b9a5c127&v=4
